In [1]:
from src.value import Value
import src.function as F
import numpy as np

In [2]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X, y = load_breast_cancer(return_X_y = True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = Value(X_train, "x_train", requires_grad=False)
X_test = Value(X_test, "x_test", requires_grad=False)
y_train = Value(y_train, "y_train", requires_grad=False)
y_test = Value(y_test, "y_test", requires_grad=False)

In [3]:
w = Value(np.random.randn(X.shape[1]) * 10**(-2), "w", requires_grad = True)
b = Value([0.0], "b", requires_grad = True)

In [4]:
lr = 0.01

matmul1 = F._FunctionFactory().get_new_function_of_type(F._Matmul)
add1 = F._FunctionFactory().get_new_function_of_type(F._Add)
loss = F._FunctionFactory().get_new_function_of_type(F._BCELossWithLogits)
    
def fit(num_epoch):     
    for i in range(num_epoch):
        
        y = add1.forward(matmul1.forward(X_train, w), b)
        l = loss.forward(y, y_train)
        loss.backward()
        w.value -= lr * w.grad
        b.value -= lr * b.grad
        l.zero_grad()   
        
        # if i % 10:
        print(f"Epoch {i+1}/{num_epoch}: Train Loss: {l.value}")

def predict(X):
    y = add1.forward(matmul1.forward(X, w), b)
    y_pred = F.sigmoid(y).value > 0.5
    
    return y_pred

Creating Function of type <class 'src.function._Matmul'> with name matmul_0
Creating Function of type <class 'src.function._Add'> with name add_0
Creating Function of type <class 'src.function._BCELossWithLogits'> with name bce_loss_logit_0


In [5]:
fit(100)    

Epoch 1/100: Train Loss: 0.6900616480942278
Epoch 2/100: Train Loss: 0.38183126977025655
Epoch 3/100: Train Loss: 0.20690998694325233
Epoch 4/100: Train Loss: 0.13204083009061715
Epoch 5/100: Train Loss: 0.11339020508807555
Epoch 6/100: Train Loss: 0.10256951880652228
Epoch 7/100: Train Loss: 0.09614503144070978
Epoch 8/100: Train Loss: 0.09169547931190815
Epoch 9/100: Train Loss: 0.0882365151401027
Epoch 10/100: Train Loss: 0.08536802302285386
Epoch 11/100: Train Loss: 0.08289853795534625
Epoch 12/100: Train Loss: 0.08072153891839201
Epoch 13/100: Train Loss: 0.07877142052169063
Epoch 14/100: Train Loss: 0.07700517275379616
Epoch 15/100: Train Loss: 0.07539323825877714
Epoch 16/100: Train Loss: 0.0739143772670008
Epoch 17/100: Train Loss: 0.07255265143493485
Epoch 18/100: Train Loss: 0.071295619816798
Epoch 19/100: Train Loss: 0.07013323348309923
Epoch 20/100: Train Loss: 0.06905713072419592
Epoch 21/100: Train Loss: 0.06806016363817086
Epoch 22/100: Train Loss: 0.06713606254489667
Ep

In [6]:
y_train_pred = predict(X_train)
y_test_pred = predict(X_test)

Creating Function of type <class 'src.function._Sigmoid'> with name sigmoid_0
Creating Function of type <class 'src.function._Sigmoid'> with name sigmoid_1


In [7]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, auc, roc_curve, matthews_corrcoef

def print_metrics(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    fpr, tpr, thresholds = roc_curve(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)    

    print(f"Confusion Matrix")
    print(f"{conf_matrix}\n")
    print(f"Accuracy: {(tp + tn)/(tp + tn + fn + fp)}")
    print(f"Precision: {tp / (tp + fp)}")
    print(f"Recall: {tp / (tp + fn)}")
    print(f"AUC score: {auc(fpr, tpr)}")
    print(f"MCC score: {mcc}")
    print(f"F1-Score: {(tp)/(tp+(fp+fn)/2)}\n")
    
print_metrics(y_train.value, y_train_pred)
print("")
print_metrics(y_test.value, y_test_pred)

Confusion Matrix
[[157   4]
 [  1 264]]

Accuracy: 0.9882629107981221
Precision: 0.9850746268656716
Recall: 0.9962264150943396
AUC score: 0.9856908472987226
MCC score: 0.9750566521699535
F1-Score: 0.9906191369606003


Confusion Matrix
[[50  1]
 [ 0 92]]

Accuracy: 0.993006993006993
Precision: 0.989247311827957
Recall: 1.0
AUC score: 0.9901960784313725
MCC score: 0.9848097814876146
F1-Score: 0.9945945945945946

